### This notebook has been run multiple times to ensure the results in the paper

In [1]:
from datasets import load_dataset


dataset = load_dataset("csv", data_files="challenge_data/snips/full_snips_gpt_filt.csv", split='train', keep_default_na=False)

dct_dataset = dataset.train_test_split(test_size=0.8) # this changes based on the dataset

dataset_train = dct_dataset['train']
dataset_test = dct_dataset['test']

/opt/anacondas/anaconda3/envs/EMNLP/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Found cached dataset csv (/home/jan_cegin/.cache/huggingface/datasets/csv/default-8e04aeeb1f287e7e/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)


In [3]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset_train.map(tokenize_function, batched=True)
tokenized_test_datasets = dataset_test.map(tokenize_function, batched=True)

In [4]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-large-uncased", num_labels=7)

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint a

In [5]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="bert_taboo")

import numpy as np
import evaluate

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="bert_taboo", evaluation_strategy="no", save_strategy="no", per_device_eval_batch_size=32, per_device_train_batch_size=4, learning_rate=1e-5, num_train_epochs=5, save_steps=3000)

In [6]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    compute_metrics=compute_metrics
)

In [7]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/opt/anacondas/anaconda3/envs/EMNLP/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 4079
  Num Epochs = 5
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 5100
  Number of trainable parameters = 335149063


Step,Training Loss
500,0.689900
1000,0.070900
1500,0.013900
2000,0.028300
2500,0.002900
3000,0.004400
3500,0.000400
4000,0.000100
4500,0.000100
5000,0.000100




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=5100, training_loss=0.07951501930936002, metrics={'train_runtime': 1952.7534, 'train_samples_per_second': 10.444, 'train_steps_per_second': 2.612, 'total_flos': 1.900706125395456e+16, 'train_loss': 0.07951501930936002, 'epoch': 5.0})

#### evaluating on different data

In [8]:
#trainer.evaluate(tokenized_test_datasets)

In [9]:
dataset_orig_test = load_dataset("csv", data_files="challenge_data/snips/full_snips_orig_test.csv", split='train', keep_default_na=False)


dataset = load_dataset("csv", data_files="challenge_data/snips/full_snips_human.csv", split='train', keep_default_na=False)

dct_dataset = dataset.train_test_split(test_size=0.2)

dataset_human_test = dct_dataset['test']


dataset = load_dataset("csv", data_files="challenge_data/snips/full_snips_gpt.csv", split='train', keep_default_na=False)

dct_dataset = dataset.train_test_split(test_size=0.2)

dataset_gpt_test = dct_dataset['test']



dataset = load_dataset("csv", data_files="challenge_data/snips/full_snips_gpt_filt.csv", split='train', keep_default_na=False)

dct_dataset = dataset.train_test_split(test_size=0.2)

dataset_gpt_test_filt = dct_dataset['test']

Found cached dataset csv (/home/jan_cegin/.cache/huggingface/datasets/csv/default-b89a809547fd631b/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
Found cached dataset csv (/home/jan_cegin/.cache/huggingface/datasets/csv/default-afea9912fcaad8fc/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
Loading cached split indices for dataset at /home/jan_cegin/.cache/huggingface/datasets/csv/default-afea9912fcaad8fc/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-6051a40eb1e2255b.arrow and /home/jan_cegin/.cache/huggingface/datasets/csv/default-afea9912fcaad8fc/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-784a7edaab067296.arrow
Found cached dataset csv (/home/jan_cegin/.cache/huggingface/datasets/csv/default-49d72d8f63e9e4aa/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
Loading cached split indices for dataset at /home/jan_cegin/.cache/huggingface/datasets/csv/d

In [10]:
tokenized_dataset_orig_t = dataset_orig_test.map(tokenize_function, batched=True)
tokenized_dataset_h_t= dataset_human_test.map(tokenize_function, batched=True)
tokenized_dataset_gpt = dataset_gpt_test.map(tokenize_function, batched=True)
tokenized_dataset_gpt_filt = dataset_gpt_test_filt.map(tokenize_function, batched=True)

Loading cached processed dataset at /home/jan_cegin/.cache/huggingface/datasets/csv/default-b89a809547fd631b/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-3ce194e061975e12.arrow
Loading cached processed dataset at /home/jan_cegin/.cache/huggingface/datasets/csv/default-afea9912fcaad8fc/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-06a3f89484b6711a.arrow
Loading cached processed dataset at /home/jan_cegin/.cache/huggingface/datasets/csv/default-49d72d8f63e9e4aa/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-78704634e32b2d34.arrow
Loading cached processed dataset at /home/jan_cegin/.cache/huggingface/datasets/csv/default-8e04aeeb1f287e7e/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-1e1a8c65a603daca.arrow


In [11]:
trainer.evaluate(tokenized_dataset_orig_t)

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 697
  Batch size = 32


{'eval_loss': 0.02816127985715866,
 'eval_accuracy': 0.9928263988522238,
 'eval_runtime': 15.5689,
 'eval_samples_per_second': 44.769,
 'eval_steps_per_second': 1.413,
 'epoch': 5.0}

In [12]:
trainer.evaluate(tokenized_dataset_h_t)

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 826
  Batch size = 32


{'eval_loss': 0.22659948468208313,
 'eval_accuracy': 0.9673123486682809,
 'eval_runtime': 18.461,
 'eval_samples_per_second': 44.743,
 'eval_steps_per_second': 1.408,
 'epoch': 5.0}

In [13]:
#trainer.evaluate(tokenized_dataset_gpt)

In [14]:
trainer.evaluate(tokenized_dataset_gpt_filt)

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 4080
  Batch size = 32


{'eval_loss': 0.018339693546295166,
 'eval_accuracy': 0.9970588235294118,
 'eval_runtime': 91.1841,
 'eval_samples_per_second': 44.745,
 'eval_steps_per_second': 1.404,
 'epoch': 5.0}